<a href="https://colab.research.google.com/github/sungshin-ai/ai_teamplay/blob/main/Inception_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. 모듈

In [ ]:
import os
import glob
from pathlib import Path

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
import math

import pickle
import cv2

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models, layers, Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input #InceptionV3, Xception, InceptionResNetV2
#from keras.applications.imagenet_utils import preprocess_input #others
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization

from tensorflow import keras
from PIL import Image
import tensorflow as tf

from keras.callbacks import ModelCheckpoint

## 1. 데이터

In [ ]:
select_df['path']='/content/drive/MyDrive/KSL_rgb/'+select_df['name']

In [ ]:
select_df=pd.read_csv('/content/drive/MyDrive/DF_select.csv')
select_df

In [ ]:
datagen = ImageDataGenerator(rescale=1. / 255, validation_split=0.25)
train_generator = datagen.flow_from_dataframe(
          dataframe=select_df,
          batch_size=128,
          class_mode='categorical',
          shuffle=True,
          x_col='path',
          y_col='label',
          subset = 'training')

valid_generator = datagen.flow_from_dataframe(
          dataframe=select_df,
          batch_size=128,
          class_mode='categorical',
          shuffle=True,
          x_col='path',
          y_col='label',
          subset = 'validation')

# InceptionV3

In [ ]:
InceptionV3(
    include_top=True, # classification (softmax) 부분을 포함할 것인지 선택하는 것입니다.
    weights="imagenet", # imageNet으로 pre-trained 된 모델을 받을 것인지를 선택하는 것입니다. 만약 pre-trained 된 모델을 받길 원하지 않는다면 None으로 넣으면 됩니다.
    input_tensor=None, 
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)

## 2. 모델 학습/검증

In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False, pooling='avg') #Downloading data from https://github.com/fchollet/deep-learning-models/releases/download/v0.5/inception_v3_weights_tf_dim_ordering_tf_kernels.h5

#base_model.summary()

net = Dense(units=10)(base_model.output)
net = Activation('softmax')(net)
model = Model(inputs=base_model.input, outputs=net)

model.summary()

In [ ]:
checkpoint_filepath = '/content/drive/MyDrive/keras/checkpoint/best_inc.ckpt'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1)

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

In [ ]:
history = model.fit(
          train_generator,
          epochs=10,
          validation_data=valid_generator, 
          callbacks=[model_checkpoint_callback]
)